# Study of how to compute partial derivatives using pyTorch's autodiff
This is needed in this context
 to compute the terms using for solving PDEs using Physics-Informed Neural Nets (PINNs).

There are two main ways to obtain the (partial) derivatives/gradients of a pyTorch function (e.g., neural net):
 * __Using `autograd.grad`__: This function works well but expects a scalar input function. The sum is used because the derivative of a sum of variable with respect to one input variable depends only on the corresponding output variable.
 * __Using the tensor `backward` method__: This needs a starting tensor of ones with the same dimensions as the output so feed/backpropagate via the `backward` method. Gradients can be accessed on the `.grad` attribute.
    * How to get second derivatives? <del datetime="2021-09-21">_Major limitation?_</del>/><br/> Apply the `backward` method on the first derivative!
    * Also requires more careful initialization and bookkeeping because we need to ensure `requires_grad=True` for the input tensors wrt to which one will need the derivatives. 

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import autograd

In [4]:
# base model class to test approaches

class TestNet(nn.Module):
    def __init__(self):
        super().__init__()
        self._layers = [nn.Linear(2, 5),
                        nn.Linear(5, 1)]

    def forward(self, x, t):
        xx = torch.cat([x, t], axis=1)
        h = torch.sigmoid(self._layers[0](xx))
        y = self._layers[1](h)
        return y


xg, tg = np.meshgrid(np.linspace(-1, 1, 5), np.linspace(0, 1, 7))
x = torch.as_tensor(xg.reshape((-1,1))).float().requires_grad_(True)
t = torch.as_tensor(tg.reshape((-1,1))).float().requires_grad_(True)

net = TestNet()
u = net(x, t)
print(u.shape)

torch.Size([35, 1])


### Using `autograd.grad` explicitly (on the sum)

In [5]:
# first-order derivatives
ux = autograd.grad(u.sum(), x, create_graph=True)[0]  # du/dx
ut = autograd.grad(u.sum(), t, create_graph=True)[0]  # du/dx
print('du/dx =', ux.T)
print('du/dt =', ut.T)

du/dx = tensor([[-0.0807, -0.0830, -0.0814, -0.0763, -0.0689, -0.0810, -0.0829, -0.0808,
         -0.0754, -0.0677, -0.0812, -0.0827, -0.0802, -0.0744, -0.0664, -0.0814,
         -0.0824, -0.0795, -0.0733, -0.0652, -0.0815, -0.0820, -0.0787, -0.0722,
         -0.0638, -0.0815, -0.0816, -0.0778, -0.0710, -0.0625, -0.0814, -0.0810,
         -0.0768, -0.0697, -0.0611]], grad_fn=<PermuteBackward>)
du/dt = tensor([[0.0068, 0.0064, 0.0073, 0.0095, 0.0126, 0.0057, 0.0056, 0.0067, 0.0091,
         0.0124, 0.0046, 0.0047, 0.0061, 0.0088, 0.0123, 0.0036, 0.0039, 0.0056,
         0.0084, 0.0121, 0.0026, 0.0031, 0.0050, 0.0081, 0.0119, 0.0016, 0.0023,
         0.0045, 0.0077, 0.0117, 0.0006, 0.0016, 0.0040, 0.0074, 0.0115]],
       grad_fn=<PermuteBackward>)


In [6]:
# second-order derivatives
uxx = autograd.grad(ux.sum(), x, create_graph=True)[0]  # d^2u/dx^2
print('d^2u/dx^2 =', uxx.T)

d^2u/dx^2 = tensor([[-0.0086, -0.0007,  0.0070,  0.0130,  0.0166, -0.0078,  0.0002,  0.0078,
          0.0136,  0.0169, -0.0070,  0.0011,  0.0086,  0.0142,  0.0172, -0.0061,
          0.0020,  0.0094,  0.0147,  0.0175, -0.0052,  0.0030,  0.0102,  0.0153,
          0.0177, -0.0043,  0.0039,  0.0110,  0.0158,  0.0179, -0.0033,  0.0048,
          0.0117,  0.0162,  0.0181]], grad_fn=<PermuteBackward>)


### Using tensor's `backward` method

In [7]:
# first-order derivatives
x.grad = None  # need to "zero-out" gradients on the vars of interest
t.grad = None
u.backward(gradient=torch.ones_like(u), create_graph=True)
print('du/dx =', x.grad.T)
print('du/dt =', t.grad.T)

du/dx = tensor([[-0.0807, -0.0830, -0.0814, -0.0763, -0.0689, -0.0810, -0.0829, -0.0808,
         -0.0754, -0.0677, -0.0812, -0.0827, -0.0802, -0.0744, -0.0664, -0.0814,
         -0.0824, -0.0795, -0.0733, -0.0652, -0.0815, -0.0820, -0.0787, -0.0722,
         -0.0638, -0.0815, -0.0816, -0.0778, -0.0710, -0.0625, -0.0814, -0.0810,
         -0.0768, -0.0697, -0.0611]], grad_fn=<PermuteBackward>)
du/dt = tensor([[0.0068, 0.0064, 0.0073, 0.0095, 0.0126, 0.0057, 0.0056, 0.0067, 0.0091,
         0.0124, 0.0046, 0.0047, 0.0061, 0.0088, 0.0123, 0.0036, 0.0039, 0.0056,
         0.0084, 0.0121, 0.0026, 0.0031, 0.0050, 0.0081, 0.0119, 0.0016, 0.0023,
         0.0045, 0.0077, 0.0117, 0.0006, 0.0016, 0.0040, 0.0074, 0.0115]],
       grad_fn=<PermuteBackward>)


In [8]:
# second-order derivatives
x.grad = None
ux.backward(gradient=torch.ones_like(ux), create_graph=True)
print('d^2u/dx^2 =', x.grad.T)

d^2u/dx^2 = tensor([[-0.0086, -0.0007,  0.0070,  0.0130,  0.0166, -0.0078,  0.0002,  0.0078,
          0.0136,  0.0169, -0.0070,  0.0011,  0.0086,  0.0142,  0.0172, -0.0061,
          0.0020,  0.0094,  0.0147,  0.0175, -0.0052,  0.0030,  0.0102,  0.0153,
          0.0177, -0.0043,  0.0039,  0.0110,  0.0158,  0.0179, -0.0033,  0.0048,
          0.0117,  0.0162,  0.0181]], grad_fn=<PermuteBackward>)
